In [9]:
import numpy as np
import random

# pretty print for np.array 
# from https://stackoverflow.com/questions/53126305/pretty-printing-numpy-ndarrays-using-unicode-characters/53164538#53164538
def pretty_print(A):
    if A.ndim==1:
        print(A)
    else:
        w = max([len(str(s)) for s in A]) 
        print(u'\u250c' + u'\u2500' * w + u'\u2510') 
        for AA in A:
            print(' ', end='')
            print('[', end='')
            for i, AAA in enumerate(AA[:-1]):
                w1 = max([len(str(s)) for s in A[:, i]])
                print(str(AAA) + ' ' * (w1  - len(str(AAA)) + 1), end='')
            w1 = max([len(str(s)) for s in A[:, -1]])
            print(str(AA[-1]) + ' ' * (w1 - len(str(AA[-1]))), end='')
            print(']')
        print(u'\u2514'+u'\u2500' * w + u'\u2518')  

### Generate $\mathcal{A}_n^2$

> Let $|\cdot|$ denotes cardinality and $\Delta$ denote symmetric different

In [10]:
def Delta(left: set, right: set):
    return left.symmetric_difference(right)

def Cardi(x: set):
    return len(x)

def legal(alpha_i: set, alpha_i_1: set):
    return Cardi(alpha_i) <= Cardi(alpha_i_1) and len(Delta(alpha_i, alpha_i_1)) <= 2

> Let $n=2^m$, $\Omega$ be a set of $m$ element such that $|\alpha_i| \leq |\alpha_{i+1}|$ and $|\alpha_i \Delta \alpha_{i+1}| \leq 2$  
> Let $\alpha_0=\{\varnothing\}$, now genereate $\Omega$

In [17]:
# param
m = 4
# generate \Omega
max_step = 5
omega = [set([]), set([1])]
# 0 -> escape;1 -> add; 2 -> replace; 3 -> delete
for i in range(1, 2**m - 1):
    if (len(omega[i]) == m):
        omega.append(omega[i])
        continue
    alpha = omega[i].copy()
    alpha_changed = omega[i].copy()
    step = 0
    while(step <= max_step):
        strategy = random.randint(0, 3) if len(alpha) > len(omega[i]) else random.randint(0, 2)
        step += 1
        if (step == max_step):
            omega.append(alpha)
            break

        if strategy == 0:
            omega.append(alpha)
            break
        elif strategy == 1:
            alpha_changed.add(max(alpha) + 1)
        elif strategy == 2:
            replacer = max(alpha) + 1
            to_remove = random.sample(list(alpha), 1)[0]
            alpha_changed.remove(to_remove)
            alpha_changed.add(replacer)
        elif strategy == 3:
            to_remove = random.sample(list(alpha), 1)[0]
            alpha_changed.remove(to_remove)
        
        if not legal(omega[i], alpha_changed):
            omega.append(alpha)
            break
        else:
            alpha = alpha_changed.copy()

for i in range(m - 1):
    assert(legal(omega[i], omega[i + 1]))
assert(len(omega) == 2**m)

omega

[set(),
 {1},
 {1},
 {1, 4},
 {1, 4},
 {1, 4, 5},
 {1, 4, 5},
 {1, 4, 5, 6, 7},
 {1, 4, 5, 6, 7},
 {1, 4, 5, 6, 7},
 {1, 4, 5, 6, 7},
 {1, 5, 6, 7, 8},
 {1, 5, 6, 7, 8, 9},
 {1, 5, 6, 7, 8, 9},
 {1, 5, 6, 7, 8, 10},
 {1, 5, 6, 7, 8, 10}]

> $$a_{ij} = \begin{cases}
    -1,\;\alpha_j\bigcap(\alpha_{i-1}\bigcup\alpha_i)=\alpha_{i-1}\Delta\alpha_{i}\;and\;|\alpha_{i-1}\Delta\alpha_{i}|=2 \\
    (-1)^{|\alpha_{i-1}\bigcap\alpha_i| + 1},\;\alpha_{j}\bigcap(\alpha_{i-1}\bigcup\alpha_{i})\neq\varnothing\;but\;does\;not\;meet\;the\;condition\;above \\
    1,\;\alpha_j\bigcap(\alpha_{i-1}\bigcup\alpha_i)=\varnothing \\
\end{cases}
$$

In [26]:
def query_element(i: int, j: int, omega: list) -> int:
    alpha_j = omega[j]
    alpha_i_1 = omega[i - 1]
    alpha_i = omega[i]

    if alpha_j.intersection(alpha_i_1.union(alpha_i)) == Delta(alpha_i_1, alpha_i) \
        and Cardi(Delta(alpha_i_1, alpha_i)) == 2:
        return -1
    elif Cardi(alpha_j.intersection(alpha_i_1.union(alpha_i))) != 0:
        return (-1)**(Cardi(alpha_i_1.intersection(alpha_i)) + 1)
    elif Cardi(alpha_j.intersection(alpha_i_1.union(alpha_i))) == 0:
        return 1
    else:
        raise ValueError("Undefined behavior!")
    
A_mat = np.zeros((2**m, 2**m))
for i in range(A_mat.shape[0]):
    for j in range(A_mat.shape[1]):
        A_mat[i, j] = query_element(i, j, omega)

pretty_print(A_mat.astype(int))

┌─────────────────────────────────────────────────┐
 [1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [1 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ]
 [1 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ]
 [1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [1 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ]
 [1 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ]
 [1 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ]
 [1 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ]
 [1 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ]
 [1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [1 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ]
 [1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [1 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ]
 [1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
└─────────────────────────────────────────────────┘


### Generate $\mathcal{A}_{n - 1}^1$

> Let $B\in\mathcal{A}_{n - 1}^1$, $\Phi(B)\in\mathcal{A}_{n}^2\;$:
> $$ \Phi(B) = \left(\begin{array}{cc} 
1 & 1_{n-1}\\
-1^(T)_{n-1} & 2B-J_{n-1}
\end{array}\right)
> $$
> Thus: 
> $$ \Phi(B)^{-1} = \left(\begin{array}{cc} 
1-\frac{1}{2}1_{n-1}B^{-1}1^T_{n-1} & -\frac{1}{2}1_{n-1}B^{-1}\\
\frac{1}{2}B^{-1}1^T_{n-1} & \frac{1}{2}B^{-1}
\end{array}\right)
> $$

In [28]:
def rev_Phi(B):
    B_mat = np.matrix(B)
    B_mat_I = B_mat.I
    n = B.shape[0]
    ones = np.ones((1, n - 1))
    left_top = 1 - 0.5 * ones * B_mat_I * ones.T
    right_top = -0.5 * ones * B_mat_I
    left_btn = 0.5 * B_mat_I * ones.T
    right_btn = 0.5 * B_mat_I
    top = np.concatenate((left_top, right_top), axis=1)
    btn = np.concatenate((left_btn, right_btn), axis=1)
    mat = np.concatenate((top, btn), axis=0)

    assert(mat * B == np.eye(n))
    return mat

rev_Phi(A_mat)

LinAlgError: Singular matrix

In [7]:
# concatenate test
A = np.matrix([[1, 1], [1, 1]])
B = np.matrix([[2, 2], [2, 2]])
C = np.matrix([[3, 3], [3, 3]])
D = np.matrix([[4, 4], [4, 4]])
top = np.concatenate((A, B), axis=1)
btn = np.concatenate((C, D), axis=1)
mat = np.concatenate((top, btn), axis=0)
mat

matrix([[1, 1, 2, 2],
        [1, 1, 2, 2],
        [3, 3, 4, 4],
        [3, 3, 4, 4]])